### In this notebook

* [Uploading Sample ID-s to Gaia Archive](#section1)
* [Crossmatching IDs with 2MASS](#section2)
* [Creating and Splitting VOTables](#section3)
* [Merging Returned 2MASS Chunks](#section4)
* [2MASS Crossmatching Sanity Checks](#section5)
* [GAIA@AIP Query Example Using API Token](#section6)
* [Fouesneau Catalog TAP Query](#section7)


In [ ]:
from astroquery.gaia import Gaia
import warnings

# Comment this out if you want to see warnings
warnings.filterwarnings('ignore')
import astropy
import requests
import pyvo as vo
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import os
import requests
import sys
sys.path.append("../gaia_tools/")
import data_analysis
import covariance_generation as cov
from import_functions import import_data

In [ ]:
my_path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
icrs_data = import_data(path = my_path, is_bayes = True, debug = True)

# Load Public Gaia Tables

In [ ]:
gaia_tables = Gaia.load_tables(only_names=True, include_shared_tables=True)
for table in gaia_tables:
    print(table.get_qualified_name())

# Upload Sample ID-s to Gaia <a class="anchor" id="section1"></a>   
I upload the 'source_id'-s of our sample to Gaia Archive to use them for crossmatching.

In [ ]:
my_path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"

# Import the ICRS data
icrs_data = import_data(path = my_path, is_bayes = True, debug = True)

In [ ]:
from astropy.table import Table
up_table = Table.from_pandas(icrs_data[['source_id']])
Gaia.login()

# This ran for 8min
job = Gaia.upload_table(up_table, table_name='rv_sample_source_id', verbose=True)

---

# Crossmatching with 2MASS <a class="anchor" id="section2"></a>   
I use the uploaded 'source_id'-s of our sample to crossmatch from 'tmass_best_neighbour'.

In [ ]:
rv_table_name = 'user_spoder.rv_sample_source_id'
xmatch_table_name = 'gaiadr2.tmass_best_neighbour'
query_string = 'select ALL t.source_id, t.tmass_oid FROM ' + xmatch_table_name + ' as t, ' + rv_table_name + ' as s '\
                'WHERE t.source_id = s.source_id'

In [ ]:
Gaia.login()

job_name = 'tmass_crossmatch'
outpath = '/scratch/sven/gaia_downloads/crossmatche_tmass_IDs.csv'

job = Gaia.launch_job_async(query_string,
                            name=job_name,
                            output_file=outpath,
                            output_format='csv', 
                            dump_to_file=True, 
                            verbose=True)
results = job.get_results()

---

# Convert to VOTable and Split <a class="anchor" id="section3"></a>  
I convert the returned 2MASS ID-s to a VOTable and split it into chunks to  
match GAIA@AIP requirements.

In [ ]:
import pandas as pd
str_path = '/scratch/sven/gaia_downloads/crossmatche_tmass_IDs.csv'
id_df = pd.read_csv(str_path)

In [ ]:
num_chunks = 6
chunk_size = int(len(id_df)/num_chunks)+1

for g, df in id_df.groupby(np.arange(len(id_df))//chunk_size):
    print(g)
    print(df.shape)

    t = Table.from_pandas(df)
    id_votable = astropy.io.votable.from_table(t)
    astropy.io.votable.writeto(id_votable, '/scratch/sven/gaia_downloads/id_votable_chunk_{}'.format(g))

---

# Merge Returned 2MASS Data Chunks <a class="anchor" id="section4"></a>  
I merge the 2MASS data chunks and save them as a new .csv file

In [ ]:
import pandas as pd
import glob
import os

chunks = os.path.join('/hdfs/local/sven/gaia_tools_data/crossmatched_tmass_data/', 'tmass_chunk_*.csv')
chunks = glob.glob(chunks)

merged_chunks = pd.concat(map(pd.read_csv, chunks), ignore_index=True)

In [ ]:
merged_df_path = '/hdfs/local/sven/gaia_tools_data/crossmatched_tmass_data/crossmatched_tmass_data.csv'
merged_chunks.to_csv(merged_df_path, index=False)

---

# 2MASS Crossmatch Sanity Checks <a class="anchor" id="section5"></a>  

* The ID table returned from Gaia archive and 2MASS data must have same length

In [ ]:
id_table_path = '/scratch/sven/gaia_downloads/crossmatche_tmass_IDs.csv'
crossmatch_id_table = pd.read_csv(id_table_path)

tmass_data_path = '/hdfs/local/sven/gaia_tools_data/crossmatched_tmass_data/crossmatched_tmass_data.csv'
crossmatched_tmass_data = pd.read_csv(tmass_data_path)

In [ ]:
print(crossmatch_id_table.shape)
print(crossmatched_tmass_data.shape)

* Comparing right ascension ($\alpha$) and declination ($\delta$) coordinates

In [ ]:
# Original sample
my_path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
icrs_data = import_data(path = my_path, is_bayes = True, debug = False)

In [ ]:
merged_data = icrs_data.merge(crossmatched_tmass_data, on='source_id', suffixes=('_gaia', '_tmass'))

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
plt.scatter(merged_data.ra_gaia, merged_data.ra_tmass, alpha=0.3, s = 0.7)
plt.xlabel('RA (GAIA)', fontdict={'fontsize': 15}, labelpad=10)
plt.ylabel('RA (TMASS)', fontdict={'fontsize': 15}, labelpad=10)

# inset axes....
axins = ax.inset_axes([0.5, 0.1, 0.47, 0.47])
axins.scatter(merged_data.ra_gaia, merged_data.ra_tmass, alpha=0.8, s = 0.7)

# sub region of the original image
x1, x2, y1, y2 = 99.99, 100, 99.99, 100
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticklabels([])

ax.indicate_inset_zoom(axins, edgecolor="black")

fig_name = 'ra_sanity_check'
plt.savefig('/home/sven/repos/gaia-tools/out/crossmatch_sanity_checks/' + fig_name +'.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
plt.scatter(merged_data.dec_gaia, merged_data.dec_tmass, alpha=0.3, s = 0.7)
plt.xlabel('DEC (GAIA)', fontdict={'fontsize': 15}, labelpad=10)
plt.ylabel('DEC (TMASS)', fontdict={'fontsize': 15}, labelpad=10)

# inset axes....
axins = ax.inset_axes([0.5, 0.1, 0.47, 0.47])
axins.scatter(merged_data.ra_gaia, merged_data.ra_tmass, alpha=0.8, s = 0.7)

# sub region of the original image
x1, x2, y1, y2 = 24.99, 25, 24.99, 25
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticklabels([])

ax.indicate_inset_zoom(axins, edgecolor="black")

fig_name = 'dec_sanity_check'
plt.savefig('/home/sven/repos/gaia-tools/out/crossmatch_sanity_checks/' + fig_name +'.png', dpi=300)

---

# GAIA@AIP Query Example Using API Token <a class="anchor" id="section6"></a>  

In [ ]:
import requests
import pyvo as vo

name = 'GAIA@AIP'
url = 'https://gaia.aip.de/tap'
token = '9ef51a3b42860269f9cc7d5e2fa90cf026fc0815'
starhorse_string = 'SELECT TOP 50 * \
FROM gaiaedr3_contrib.starhorse'

print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Setup authorization
tap_session = requests.Session()
tap_session.headers['Authorization'] = token

tap_service = vo.dal.TAPService(url, session=tap_session)

tap_result = tap_service.run_async(starhorse_string)
tap_result.to_table()

print(tap_result)

In [ ]:
output_df = tap_result.to_table().to_pandas()
output_df.to_csv('test.csv')

---
# Fouesneau Catalog TAP Query <a class="anchor" id="section7"></a>  

In [ ]:
url_heidelberg = "http://dc.zah.uni-heidelberg.de/__system__/tap/run" 
heidelberg_string = "select all g.source_id from gdr2ap.main as g where g.source_id = {}"

# Setup authorization
tap_session = requests.Session()

tap_service = vo.dal.TAPService(url_heidelberg)
print('Maxrec {}'.format(tap_service.maxrec))
print('Hardlimit {}'.format(tap_service.hardlimit))

tap_result = tap_service.run_async(heidelberg_string.format(3), maxrec=10000000)

for id in icrs_data.source_id[0:5]:
    print(id)
    tap_result = tap_service.run_async(heidelberg_string.format(id), maxrec=10000000)

---